# Normalize Micropal CSVs

In [1]:
import sys
sys.path.append('../scripts/')
import glob
import re
import os.path

import pandas as pd
import numpy as np

from normalize_data import (
    csv_cleanup,
    update_metadata,
    get_taxonomy_columns,
    clean_taxon_name
)

In [2]:
taxa_list_path = 'cleaned_data/taxa_list.csv'
non_taxa_fields_path = 'cleaned_data/non_taxa_fields.csv'

In [3]:
micropal_1 = 'cleaned_data/Micropal_CSV_1'
micropal_meta_1 = 'cleaned_data/metadata/Micropal_1_changes.csv'

micropal_2 = 'cleaned_data/Micropal_CSV_2'
micropal_meta_2 = 'cleaned_data/metadata/Micropal_2_changes.csv'

micropal_3 = 'cleaned_data/Micropal_CSV_3'
micropal_meta_3 = 'cleaned_data/metadata/Micropal_3_changes.csv'

In [16]:
metadata_file = micropal_meta_3
clean_data_path = micropal_3

In [17]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,fix_expedition_aw_cols,add_extra_sample_data
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False,False,True,False,False,False,False
1,324_U1348A_benthic_forams.csv,benthic_forams,False,True,True,False,True,False,False
2,339_planktic_forams_U1387C.csv,planktic_forams,False,False,True,False,False,False,False
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False,False,True,False,False,False,False
4,341_radiolarians_U1419D.csv,radiolarians,True,False,False,False,True,True,False


In [18]:
non_taxa_content = pd.read_csv(non_taxa_fields_path)
non_taxa_columns = set(non_taxa_content['field'])

In [19]:
taxa_metadata_columns = set(non_taxa_content[non_taxa_content['type'] == 'taxonomy metadata']['field'])

## Clean up taxa values

Look for taxa columns that have "code [extra text]", and remove "[extra text]".

In [7]:
def clean_up_taxa_values(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    taxa_cols = get_taxonomy_columns(content.columns, non_taxa_columns)            
    taxa_df = content[taxa_cols]
    content[taxa_cols] = taxa_df.replace(to_replace =' *\[.*\] *', value = '', regex = True) 
    
    # get rid of NAs in order to compare two dataframes   
    changed = not taxa_df.fillna('').equals(content[taxa_cols].fillna(''))

    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [clean_up_taxa_values(file) for file in metadata['file']] 

### Update metadata

In [8]:
dict = {"clean_up_taxa_values": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False,False,True,False,False
1,324_U1348A_benthic_forams.csv,benthic_forams,False,True,True,False,True
2,339_planktic_forams_U1387C.csv,planktic_forams,False,False,True,False,False
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False,False,True,False,False
4,341_radiolarians_U1419D.csv,radiolarians,True,False,False,False,True


In [9]:
new_metadata.to_csv(metadata_file, index=False)

## Clean up taxa metadata values

Look for taxa metadata columns that have "code [extra text]", and remove "[extra text]".

In [20]:
def clean_up_taxa_meta_values(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    available_metadata = list(taxa_metadata_columns.intersection(set(content.columns)))
    
    taxa_meta_df = content[available_metadata]
    content[available_metadata] = taxa_meta_df.replace(to_replace =' *\[.*\] *', value = '', regex = True) 
    
    # get rid of NAs in order to compare two dataframes   
    changed = not taxa_meta_df.fillna('').equals(content[available_metadata].fillna(''))

    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [clean_up_taxa_meta_values(file) for file in metadata['file']] 

### Update metadata

In [21]:
dict = {"clean_up_taxa_metadata_values": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,fix_expedition_aw_cols,add_extra_sample_data,clean_up_taxa_metadata_values
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False,False,True,False,False,False,False,False
1,324_U1348A_benthic_forams.csv,benthic_forams,False,True,True,False,True,False,False,True
2,339_planktic_forams_U1387C.csv,planktic_forams,False,False,True,False,False,False,False,False
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False,False,True,False,False,False,False,False
4,341_radiolarians_U1419D.csv,radiolarians,True,False,False,False,True,True,False,True


In [22]:
new_metadata.to_csv(metadata_file, index=False)

## Create a csv of all taxa

Create a taxa list csv that contains all the taxon names and the associated taxon group.

In [10]:
taxa_data = set()

Read the existing data from the existing taxa list csv. 

In [11]:
def fetch_existing_taxa_data(row):
    taxa_data.update([(row['verbatim_name'], row['name'], row['taxon_group'])])

if os.path.isfile(taxa_list_path): 
    existing_taxa_df = pd.read_csv(taxa_list_path)
    existing_taxa_df = csv_cleanup(existing_taxa_df, taxa_list_path)
    existing_taxa_df.apply(fetch_existing_taxa_data, axis=1)
    
len(taxa_data)

3540

Get taxa data from the cleaned up taxa csv.

In [12]:
def fetch_taxa_data(row):
    path = f"{clean_data_path}/{row['file']}"
    content = pd.read_csv(path)
    content = csv_cleanup(content, path)

    content = content.dropna(axis="columns", how="all")
    columns = [col.strip() for col in content.columns if col]
    
    taxa_columns = set(columns) - non_taxa_columns 
    taxa_data.update(((col, clean_taxon_name(col), row['taxon_group']) for col in taxa_columns))

res = metadata[['file', 'taxon_group']].apply(fetch_taxa_data, axis=1)

In [13]:
len(taxa_data)

4738

### Create csv

In [14]:
taxa_df = pd.DataFrame(taxa_data, columns=['verbatim_name', 'name', 'taxon_group'])
taxa_df.sort_values('name', inplace=True)
taxa_df.head(15)

,verbatim_name,name,taxon_group
2935,"""Globigerina"" angulisuturalis _T","""Globigerina"" angulisuturalis _T",planktic_forams
983,"""Globigerina"" angulisuturalis _T_","""Globigerina"" angulisuturalis _T_",planktic_forams
3735,"""Globigerina"" ciperoensis _T","""Globigerina"" ciperoensis _T",planktic_forams
385,"""Globigerina"" ciperoensis _T_","""Globigerina"" ciperoensis _T_",planktic_forams
2617,"""Skeletonema"" utriculosa","""Skeletonema"" utriculosa",diatoms
1265,"""round browns"" indet., psilate","""round browns"" indet., psilate",dinoflagellates
4317,(Duplicate) Dentoglobigerina pseudovenezuelana,(Duplicate) Dentoglobigerina pseudovenezuelana,planktic_forams
1239,?Acarinina spp.,?Acarinina spp.,planktic_forams
3301,?Morozovella spp.,?Morozovella spp.,planktic_forams
3323,ADDITIONAL SPECIES,ADDITIONAL SPECIES,planktic_forams


In [15]:
taxa_df.to_csv(taxa_list_path, index=False)